## <center>Лабораторная работа № 7 'Классификация обзоров фильмов'<center>

### <center>Выполнила студентка 3 курса группы БФИ2001 Калмыкова Дарья<center>

### Цель
Обучение на датасете IMDB с помощью рекуррентной нейронной сети

### Задачи
* Ознакомиться с задачей рекуррентными нейронными сетями
* Изучить способы классификации текста
* Ознакомиться с ансамблированием сетей
* Построить ансамбль сетей, который позволит получать точность не менее 97%

### Требования
1. Найти набор оптимальных ИНС для классификации текста
2. Провести ансамблирование моделей
3. Написать функцию/функции, которые позволят загружать текст и получать 
результат ансамбля сетей
4. Провести тестирование сетей на своих текстах

In [1]:
import numpy as np
import re
import spacy
import gensim

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.initializers import glorot_uniform

from keras.datasets import imdb
from keras.models import Sequential
from keras.regularizers import l2
from keras.layers import Dense, LSTM, Embedding, Conv1D, MaxPooling1D, Dropout, BatchNormalization
from tensorflow.keras.saving import load_model

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
data = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

In [3]:
max_review_length = 500
embedding_vecor_length = 32
X_train = pad_sequences(X_train, maxlen=max_review_length)
X_test = pad_sequences(X_test, maxlen=max_review_length)

### Model 1

In [4]:
model_base = Sequential()

model_base.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model_base.add(LSTM(100))
model_base.add(Dense(1, activation='sigmoid'))
model_base.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model_base.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
model_base.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

Epoch 1/2
391/391 [==============================] - 340s 865ms/step - loss: 0.4668 - accuracy: 0.7717 - val_loss: 0.3522 - val_accuracy: 0.8496
Epoch 2/2
391/391 [==============================] - 352s 902ms/step - loss: 0.3047 - accuracy: 0.8757 - val_loss: 0.3133 - val_accuracy: 0.8690


In [6]:
scores = model_base.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.90%


### Model 2

In [7]:
model1 = Sequential()

model1.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model1.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2))

model1.add(LSTM(120))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           160000    
                                                                 
 conv1d (Conv1D)             (None, 500, 32)           3104      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 250, 32)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 120)               73440     
                                                                 
 dense_1 (Dense)             (None, 1)                 121       
                                                                 
Total params: 236,665
Trainable params: 236,665
Non-trainable params: 0
________________________________________________

In [9]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

Epoch 1/2
391/391 [==============================] - 156s 394ms/step - loss: 0.4584 - accuracy: 0.7676 - val_loss: 0.3009 - val_accuracy: 0.8788
Epoch 2/2
391/391 [==============================] - 148s 378ms/step - loss: 0.2470 - accuracy: 0.9038 - val_loss: 0.3055 - val_accuracy: 0.8804


In [10]:
scores = model1.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

782/782 [==============================] - 41s 52ms/step - loss: 0.3055 - accuracy: 0.8804
Accuracy: 88.04%


### Model 3

In [11]:
model2 = Sequential()

model2.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model2.add(LSTM(30))

model2.add(Dense(200, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.3))

model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [12]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 32)           160000    
                                                                 
 lstm_2 (LSTM)               (None, 30)                7560      
                                                                 
 dense_2 (Dense)             (None, 200)               6200      
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 100)               20100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 50)               

In [13]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4, batch_size=64)

Epoch 1/4
391/391 [==============================] - 87s 217ms/step - loss: 0.4362 - accuracy: 0.7846 - val_loss: 0.3747 - val_accuracy: 0.8492
Epoch 2/4
391/391 [==============================] - 84s 215ms/step - loss: 0.2613 - accuracy: 0.8986 - val_loss: 0.2949 - val_accuracy: 0.8762
Epoch 3/4
391/391 [==============================] - 83s 211ms/step - loss: 0.2231 - accuracy: 0.9147 - val_loss: 0.2975 - val_accuracy: 0.8794
Epoch 4/4
391/391 [==============================] - 83s 213ms/step - loss: 0.1878 - accuracy: 0.9303 - val_loss: 0.3116 - val_accuracy: 0.8744


In [14]:
scores = model2.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

782/782 [==============================] - 33s 42ms/step - loss: 0.3116 - accuracy: 0.8744
Accuracy: 87.44%


### Model 4

In [15]:
model3 = Sequential()

model3.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))

model3.add(Dense(250, activation='selu', kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model3.add(Dropout(0.5))
model3.add(LSTM(50))

model3.add(Dense(110, activation='selu', kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model3.add(Dropout(0.3))

model3.add(Dense(50, activation='selu', kernel_initializer=glorot_uniform(), bias_regularizer=l2(0.05)))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [16]:
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 32)           160000    
                                                                 
 dense_6 (Dense)             (None, 500, 250)          8250      
                                                                 
 dropout_2 (Dropout)         (None, 500, 250)          0         
                                                                 
 lstm_3 (LSTM)               (None, 50)                60200     
                                                                 
 dense_7 (Dense)             (None, 110)               5610      
                                                                 
 dropout_3 (Dropout)         (None, 110)               0         
                                                                 
 dense_8 (Dense)             (None, 50)               

In [17]:
model3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

Epoch 1/2
391/391 [==============================] - 221s 559ms/step - loss: 0.4483 - accuracy: 0.7787 - val_loss: 0.3194 - val_accuracy: 0.8698
Epoch 2/2
391/391 [==============================] - 220s 564ms/step - loss: 0.2818 - accuracy: 0.8885 - val_loss: 0.3130 - val_accuracy: 0.8717


In [18]:
scores = model3.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

782/782 [==============================] - 65s 83ms/step - loss: 0.3130 - accuracy: 0.8717
Accuracy: 87.17%


### Saving and loading models

In [19]:
model_base.save('model_base.keras')

In [20]:
model1.save('model1.keras')

In [21]:
model2.save('model2.keras')

In [22]:
model3.save('model3.keras')

In [23]:
loaded_model_base = load_model('model_base.keras')
loaded_model1 = load_model('model1.keras')
loaded_model2 = load_model('model2.keras')
loaded_model3 = load_model('model3.keras')

In [24]:
all_models = [loaded_model_base, loaded_model1, loaded_model2, loaded_model3]

### Stacking as Ensemble Method

In [25]:
# create stacked model input dataset as outputs from the ensemble

def stacked_dataset(all_models, X_input):
    X_stack = None
    for mdl in all_models:
        pred = mdl.predict(X_input, verbose=1)
        
        if X_stack is None:
            X_stack = pred
        else:
            X_stack = np.dstack((X_stack, pred))
    
    # flatten predictions to [rows, members x probabilities]
    X_stack = X_stack.reshape((X_stack.shape[0], X_stack.shape[1]*X_stack.shape[2]))
    return X_stack

In [27]:
def fit_stacked_model(all_models, X_train, y_train):
    X_stacked = stacked_dataset(all_models, X_train)
    
    model_gbc = GradientBoostingClassifier()
    model_gbc.fit(X_stacked, y_train)
    
    return model_gbc

In [28]:
stacked_model = fit_stacked_model(all_models, X_train, y_train)

782/782 [==============================] - 61s 78ms/step


In [29]:
def stacked_prediction(all_models, stacked_model, X_test):
    X_stacked = stacked_dataset(all_models, X_test)
    pred = stacked_model.predict(X_stacked)
    return pred

y_pred = stacked_prediction(all_models, stacked_model, X_test)
print(accuracy_score(y_test, y_pred)*100)

782/782 [==============================] - 60s 76ms/step
87.868


### Fucntion to test user's review

In [30]:
def preproc(text):
    text_punct = re.sub(r"[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]", ' ', text)
    text_numbers = re.sub(r"[0-9]", ' ', text_punct)
    
    text_token = word_tokenize(text_numbers, language='english')
    text_preproc = [word for word in text_token]
    
    return text_preproc

In [43]:
def get_index(words_prep):
    res = []
    for i in range(len(words_prep)):
        word_index = imdb.get_word_index()
        word = words_prep[i]
        if word_index[word] < 500:
            res.append(word_index[word])
    return res

### Positive review

In [67]:
text_string = str(input('Input your movie review: '))
words = preproc(text_string.lower())
words_idx = get_index(words)
words_pad = pad_sequences([words_idx], maxlen=max_review_length)

Input your movie review: When you very nearly spray a mouthful of drink over the person in front of you, its generally a good indicator the movie is pretty funny. The sandwich joke had me in stitches! This movie doesn't rely on just a few jokes to carry it, they maintain a subtle layer of humour throughout and then have you in stitches with some brilliant jokes. The cast in this movie are well picked and really gets the whole dilemma of everyday life as a vampire! Hopefully this gets picked up and the masses get a chance to enjoy this wee gem. Loved this movie and would definitely recommend it. Gave this a 10. A must see!!


In [68]:
words_stacked = stacked_dataset(all_models, words_pad)

1/1 [==============================] - 0s 47ms/step


In [73]:
pred = stacked_model.predict_proba(words_stacked)
print(pred)

[[0.02288681 0.97711319]]


In [69]:
stacked_model.score(words_stacked, [1])

1.0

### Negative review

In [78]:
text_string = str(input('Input your movie review: '))
words = preproc(text_string.lower())
words_idx = get_index(words)
words_pad = pad_sequences([words_idx], maxlen=max_review_length)

Input your movie review: Almost 3 hours wasted of my time. Garbage movie - plot wise. The plot has more holes than a Swiss cheese. Acting is okay, story though, ugh, what a poor attempt.


In [79]:
words_stacked = stacked_dataset(all_models, words_pad)


1/1 [==============================] - 0s 31ms/step


In [80]:
pred = stacked_model.predict_proba(words_stacked)
print(pred)

[[0.64621774 0.35378226]]


In [81]:
stacked_model.score(words_stacked, [0])

1.0